# Comitês de Aprendizagem


## Estudo de caso: dados sobre o censo de 1994 dos EUA
Este dataset contém dados coletados pelo censo norte americano de 1994, contendo atributos sobre cada pessoa intrevistada, sendo que o último deles indica se a pessoa possuia renda maior ou menor do que $50.000,00 anuais.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv('adult.csv')
df.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


### Separando os conjuntos de treino e teste

In [2]:
from sklearn.model_selection import train_test_split

# separando os conjuntos de dados de treino e teste
df_treino, df_teste = train_test_split(df, test_size=0.2, random_state=42)

# separando a coluna alvo do conjunto de treino
df_treino_labels = df_treino['class'].copy()
df_treino        = df_treino.drop(columns='class')

# separando a coluna alvo do conjunto de teste
df_teste_labels = df_teste['class'].copy()
df_teste        = df_teste.drop(columns='class')

### Pré-processamento

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Atributos escolhidos:
nomes_atributos_numericos   = ['age','fnlwgt','education-num','capital-gain',
                               'capital-loss','hours-per-week']
nomes_atributos_categoricos = ['workclass','education','occupation','relationship',
                               'marital-status','race','sex','native-country']

pipeline_atr_numericos = Pipeline([
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])

preproc_completo = ColumnTransformer([
    ('numericos',   pipeline_atr_numericos, nomes_atributos_numericos),
    ('categoricos', OneHotEncoder(),        nomes_atributos_categoricos),
    ], 
    sparse_threshold=0)

# pre-processamento do conjunto de treino
X_treino = preproc_completo.fit_transform(df_treino)

# pre-processamento do conjunto de teste
X_teste = preproc_completo.transform(df_teste)

In [4]:
# Coluna alvo para os conjuntos de treino e teste
y_treino = df_treino_labels.values
y_teste = df_teste_labels.values

In [5]:
y_treino

array(['>50K', '>50K', '<=50K', ..., '<=50K', '<=50K', '<=50K'],
      dtype=object)

In [6]:
X_treino.shape

(26048, 108)

In [7]:
y_treino.shape

(26048,)

In [8]:
X_teste.shape

(6513, 108)

In [9]:
y_teste.shape

(6513,)

## Regressão Logística com SKLearn

In [10]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=300)

log_reg.fit(X_treino, y_treino)
log_reg.classes_

array(['<=50K', '>50K'], dtype=object)

### Probabilidade estimada pela regressão logística

In [11]:
y_probabilidade_reglog = log_reg.predict_proba(X_teste)
np.round(y_probabilidade_reglog,decimals=2)

array([[0.98, 0.02],
       [0.55, 0.45],
       [0.33, 0.67],
       ...,
       [0.  , 1.  ],
       [0.97, 0.03],
       [0.99, 0.01]])

### Previsões pela regressão logística

In [12]:
y_previsto_reglog = log_reg.predict(X_teste)
y_previsto_reglog

array(['<=50K', '<=50K', '>50K', ..., '>50K', '<=50K', '<=50K'],
      dtype=object)

### Desempenho da regressão logística

In [13]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score   
from sklearn.metrics import recall_score      
from sklearn.metrics import f1_score          

In [14]:
print("Matriz de confusão:")
print(confusion_matrix(y_teste,y_previsto_reglog))
print()
print("Acurácia: ", accuracy_score(  y_teste,y_previsto_reglog))
print("Precisão: ", precision_score( y_teste,y_previsto_reglog,pos_label='>50K'))
print("Recall  : ", recall_score(    y_teste,y_previsto_reglog,pos_label='>50K'))
print("F1      : ", f1_score(        y_teste,y_previsto_reglog,pos_label='>50K'))

Matriz de confusão:
[[4623  319]
 [ 604  967]]

Acurácia:  0.8582834331337326
Precisão:  0.7519440124416796
Recall  :  0.6155315085932527
F1      :  0.6769338466923346


## Árvore de Decisão  com SKLearn

In [15]:
from sklearn.tree import DecisionTreeClassifier

arvore = DecisionTreeClassifier(max_depth=3)
arvore.fit(X_treino, y_treino)
arvore.classes_

array(['<=50K', '>50K'], dtype=object)

### Probabilidade estimada pela árvore de decisão

In [16]:
y_probabilidade_arvore = arvore.predict_proba(X_teste)
np.round(y_probabilidade_arvore,decimals=2)

array([[0.96, 0.04],
       [0.7 , 0.3 ],
       [0.33, 0.67],
       ...,
       [0.  , 1.  ],
       [0.96, 0.04],
       [0.96, 0.04]])

### Previsões pela árvore de decisão

In [17]:
y_previsto_arvore = arvore.predict(X_teste)
y_previsto_arvore

array(['<=50K', '<=50K', '>50K', ..., '>50K', '<=50K', '<=50K'],
      dtype=object)

### Desempenho da árvore de decisão

In [18]:
print("Matriz de confusão:")
print(confusion_matrix(y_teste,y_previsto_arvore))
print()
print("Acurácia: ", accuracy_score(  y_teste,y_previsto_arvore))
print("Precisão: ", precision_score( y_teste,y_previsto_arvore,pos_label='>50K'))
print("Recall  : ", recall_score(    y_teste,y_previsto_arvore,pos_label='>50K'))
print("F1      : ", f1_score(        y_teste,y_previsto_arvore,pos_label='>50K'))

Matriz de confusão:
[[4695  247]
 [ 761  810]]

Acurácia:  0.8452326116996776
Precisão:  0.7663197729422895
Recall  :  0.5155951623169955
F1      :  0.6164383561643835


# Comitês de Aprendizagem (Ensembles)

In [19]:
from sklearn.ensemble import VotingClassifier

logreg2 = LogisticRegression(max_iter=300)
arvore2 = DecisionTreeClassifier(max_depth=3)

votador = VotingClassifier(
                           estimators=[('log_reg',logreg2),
                                       ('arvore',arvore2),
                                      ],
                            voting='soft',
                            n_jobs=-1)

#votador.fit(X_treino, y_treino)

In [20]:
for estimador in (logreg2,arvore2,votador):
    estimador.fit(X_treino, y_treino)
    previsoes = estimador.predict(X_teste)
    print("Acurária ", estimador.__class__.__name__,np.round(accuracy_score(  y_teste, previsoes),decimals=3))

Acurária  LogisticRegression 0.858
Acurária  DecisionTreeClassifier 0.845
Acurária  VotingClassifier 0.848


In [21]:
# somente para voting='soft'
y_probabilidade_votador = votador.predict_proba(X_teste)
np.round(y_probabilidade_votador,decimals=2)

array([[0.97, 0.03],
       [0.62, 0.38],
       [0.33, 0.67],
       ...,
       [0.  , 1.  ],
       [0.97, 0.03],
       [0.98, 0.02]])

In [22]:
y_previsto_votador = votador.predict(X_teste)
y_previsto_votador

array(['<=50K', '<=50K', '>50K', ..., '>50K', '<=50K', '<=50K'],
      dtype=object)

In [23]:
print("Matriz de confusão:")
print(confusion_matrix(y_teste,y_previsto_votador))
print()
print("Acurácia: ", accuracy_score(  y_teste,y_previsto_votador))
print("Precisão: ", precision_score( y_teste,y_previsto_votador,pos_label='>50K'))
print("Recall  : ", recall_score(    y_teste,y_previsto_votador,pos_label='>50K'))
print("F1      : ", f1_score(        y_teste,y_previsto_votador,pos_label='>50K'))

Matriz de confusão:
[[4694  248]
 [ 741  830]]

Acurácia:  0.8481498541378781
Precisão:  0.7699443413729128
Recall  :  0.5283259070655634
F1      :  0.6266515666289166


## RandomForest com SKLearn

In [24]:
from sklearn.ensemble import RandomForestClassifier

rndfrst = RandomForestClassifier(n_estimators=100, max_features=0.7, max_samples=0.5, n_jobs=-1)
rndfrst.fit(X_treino, y_treino)
rndfrst.classes_

array(['<=50K', '>50K'], dtype=object)

### Probabilidade estimada por RandomForest

In [25]:
y_probabilidade_rndfrst = rndfrst.predict_proba(X_teste)
np.round(y_probabilidade_rndfrst,decimals=2)

array([[1.  , 0.  ],
       [0.64, 0.36],
       [0.45, 0.55],
       ...,
       [0.  , 1.  ],
       [1.  , 0.  ],
       [1.  , 0.  ]])

### Previsões com RandomForest

In [26]:
y_previsto_rndfrst = rndfrst.predict(X_teste)
y_previsto_rndfrst

array(['<=50K', '<=50K', '>50K', ..., '>50K', '<=50K', '<=50K'],
      dtype=object)

### Desempenho com RandomForest

In [27]:
print("Matriz de confusão:")
print(confusion_matrix(y_teste,y_previsto_rndfrst))
print()
print("Acurácia: ", accuracy_score(  y_teste,y_previsto_rndfrst))
print("Precisão: ", precision_score( y_teste,y_previsto_rndfrst,pos_label='>50K'))
print("Recall  : ", recall_score(    y_teste,y_previsto_rndfrst,pos_label='>50K'))
print("F1      : ", f1_score(        y_teste,y_previsto_rndfrst,pos_label='>50K'))

Matriz de confusão:
[[4623  319]
 [ 548 1023]]

Acurácia:  0.8668816213726394
Precisão:  0.7622950819672131
Recall  :  0.6511775938892426
F1      :  0.7023686920700308


## Bagging com SKLearn

In [28]:
from sklearn.ensemble import BaggingClassifier

log_reg_bagg = BaggingClassifier(LogisticRegression(max_iter=300),
                                 n_estimators=100, max_features=0.7, max_samples=0.5, n_jobs=-1)
log_reg_bagg.fit(X_treino, y_treino)
log_reg_bagg.classes_

array(['<=50K', '>50K'], dtype=object)

### Probabilidade estimada por Bagging de Regressão Logística

In [29]:
y_probabilidade_bagging = log_reg_bagg.predict_proba(X_teste)
np.round(y_probabilidade_bagging,decimals=2)

array([[0.96, 0.04],
       [0.59, 0.41],
       [0.33, 0.67],
       ...,
       [0.03, 0.97],
       [0.95, 0.05],
       [0.99, 0.01]])

### Previsões por Bagging de Regressão Logística

In [30]:
y_previsto_bagging = log_reg_bagg.predict(X_teste)
y_previsto_bagging

array(['<=50K', '<=50K', '>50K', ..., '>50K', '<=50K', '<=50K'],
      dtype=object)

### Desempenho com Bagging de Regressão Logística

In [31]:
print("Matriz de confusão:")
print(confusion_matrix(y_teste,y_previsto_bagging))
print()
print("Acurácia: ", accuracy_score(  y_teste,y_previsto_bagging))
print("Precisão: ", precision_score( y_teste,y_previsto_bagging,pos_label='>50K'))
print("Recall  : ", recall_score(    y_teste,y_previsto_bagging,pos_label='>50K'))
print("F1      : ", f1_score(        y_teste,y_previsto_bagging,pos_label='>50K'))

Matriz de confusão:
[[4637  305]
 [ 631  940]]

Acurácia:  0.8562874251497006
Precisão:  0.7550200803212851
Recall  :  0.5983450031826862
F1      :  0.6676136363636364
